In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import bloscpack as bp

import numpy as np

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
features_dir = '/home/yuncong/csd395/CSHL_patch_features'

In [3]:
train_features_dir = '/home/yuncong/csd395/CSHL_patch_features/train'

In [4]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

In [6]:
stack = 'MD589'

In [5]:
features_allClasses_pos = {}
features_allClasses_neg = {}

In [9]:
for label in labels:
    if label == 'BackG':
        continue
    features_allClasses_pos[label] = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_features.bp' % {'stack': stack, 'label': label})
    features_allClasses_neg[label] = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_surround_features.bp' % {'stack': stack, 'label': label})

In [5]:
from sklearn.svm import SVC
from sklearn.externals import joblib

In [6]:
stack = 'MD589'

In [9]:
os.makedirs('svm')

OSError: [Errno 17] File exists: 'svm'

In [114]:
for label in labels:
    
    if label == 'BackG':
        continue
        
    print label
    
    features_allClasses_pos = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_features.bp' % {'stack': stack, 'label': label})
    features_allClasses_neg = bp.unpack_ndarray_file(train_features_dir + '/%(stack)s_%(label)s_surround_features.bp' % {'stack': stack, 'label': label})
    
    train_data = np.r_[features_allClasses_pos, features_allClasses_neg]
    train_labels = np.r_[np.ones((features_allClasses_pos.shape[0], )), 
                         -np.ones((features_allClasses_neg.shape[0], ))]
    
    svc = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
          probability=True, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
          max_iter=-1, decision_function_shape=None, random_state=None)
    
    svc.fit(train_data, train_labels)
    
    joblib.dump(svc, 'svm/%(label)s_svm.pkl' % {'label': label})
    
    del features_allClasses_pos, features_allClasses_neg, train_data, train_labels, svc

In [6]:
svc_allClasses = {}
for label_ind, label in enumerate(labels[1:]):
    svc_allClasses[label] = joblib.load('svm/%(label)s_svm.pkl' % {'label': label})

In [7]:
predictions_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_patch_predictions_svm'
if not os.path.exists(predictions_rootdir):
    os.makedirs(predictions_rootdir)

In [31]:
def f(stack, sec):
    
    test_features_dir = features_dir + '/%(stack)s' % {'stack': stack, 'sec': sec}

#     features_roi = bp.unpack_ndarray_file(test_features_dir + '/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})
    features_roi = load_hdf(test_features_dir + '/%(stack)s_%(sec)04d_features.hdf' % {'stack': stack, 'sec': sec})

    n = features_roi.shape[0]
#     print n

#     probs = np.zeros((n, len(labels)-1))

#     for label_ind, label in enumerate(labels[1:]):

    label = '7N'
    svc = svc_allClasses[label]
#     probs[:, label_ind] = svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]

    svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]
    
#     predictions_dir = predictions_rootdir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
#     if not os.path.exists(predictions_dir):
#         os.makedirs(predictions_dir)
    
#     np.save(predictions_dir + '/%(stack)s_%(sec)04d_roi1_predictions.npy' % {'stack': stack, 'sec': sec}, probs)

In [29]:
stack = 'MD585'
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [32]:
t = time.time()
Parallel(n_jobs=16)(delayed(f)(stack=stack, sec=sec) for sec in range(first_detect_sec, first_detect_sec+16))
print time.time() - t

68.5894491673


In [138]:
stack = 'MD585'
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

# sec = 142
for sec in range(first_detect_sec, last_detect_sec+1):
    
# test_features_dir = features_dir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
    test_features_dir = features_dir + '/%(stack)s' % {'stack': stack, 'sec': sec}

#     features_roi = bp.unpack_ndarray_file(test_features_dir + '/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})
    features_roi = load_hdf(test_features_dir + '/%(stack)s_%(sec)04d_features.hdf' % {'stack': stack, 'sec': sec})

    n = features_roi.shape[0]

    probs = np.zeros((n, len(labels)-1))

    for label_ind, label in enumerate(labels[1:]):

        print label

        svc = svc_allClasses[label]
        probs[:, label_ind] = svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]

5N
7n
7N
12N


KeyboardInterrupt: 

In [33]:
stack = 'MD585'
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [36]:
for sec in range(first_detect_sec, last_detect_sec+1):
    print sec
    
# test_features_dir = features_dir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
    test_features_dir = features_dir + '/%(stack)s' % {'stack': stack, 'sec': sec}

    features_roi = bp.unpack_ndarray_file(test_features_dir + '/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})
    
    save_hdf(features_roi, test_features_dir + '/%(stack)s_%(sec)04d_features.hdf' % {'stack': stack, 'sec': sec})

132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
